<a href="https://colab.research.google.com/github/antahiap/dsr-image-dataset-curation/blob/main/Inference_with_Detectron2_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference with Detectron's object detection and segmentation models

Notebook based on the official detectron2 tutorial

https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=FsePPpwZSmqt

### Usage

* Specify the use of the GPU by going to Runtime -> Change Runtime type -> Hardware accelerator: GPU

* After installation of detectron you need you need to "restart runtime" in Colab. Runtime -> Restart Runtime

# Install detectron2

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 15.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=b8afd573257a3acd8c422d29d8e9c47f9d19eebffeb64f2481a435ab1e4f8060
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
torch:  1.12 ; cuda:  cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: 

In [ ]:
# check pytorch installation:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())


1.12.1+cu113 True


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

ModuleNotFoundError: ignored

In [ ]:
# Download a couple of images from github
!wget https://github.com/andandandand/images-for-colab-notebooks/blob/main/parked_cars.jpg?raw=true -q -O input.jpg
!wget https://raw.githubusercontent.com/andandandand/images-for-colab-notebooks/main/racing_horses.jpg -q -O input2.jpg
!wget https://raw.githubusercontent.com/andandandand/images-for-colab-notebooks/main/boxing.jpg -q -O input3.jpg
!ls

In [ ]:
# Here put your OWN image (get it from the internet somehow)
im = cv2.imread("./input2.jpg")
# cv2_imshow is a Detectron utility function for Google Colab
cv2_imshow(im)

# Run pre-trained detectron 2 models

Then, we create a detectron2 config YAML file and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
# get the YAML file with the model configuration
# https://detectron2.readthedocs.io/en/latest/modules/config.html?highlight=get_cfg#detectron2.config.get_cfg
cfg = get_cfg()
cfg

In [ ]:
# specify the model's architecture, filenames and paths are *exactly*
# the same as in https://github.com/facebookresearch/detectron2/tree/master/configs
# https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml
# notice the stem and folder at the end of the URL
# this is what should be passed to both get_config_file and get_checkpoint_url
# https://detectron2.readthedocs.io/en/latest/modules/model_zoo.html?highlight=get_config_file#detectron2.model_zoo.get_config_file
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'))
# specify the pretrained model's weights
# https://detectron2.readthedocs.io/en/latest/modules/model_zoo.html?highlight=get_checkpoint_url#detectron2.model_zoo.get_checkpoint_url
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml')

In [ ]:
#check cfg.MODEL
cfg.MODEL

In [ ]:
#check cfg.MODEL.WEIGHTS
cfg.MODEL.WEIGHTS

In [ ]:
# set the activation threshold for this model
# experiment: Check the output when making this value lower and/or higher
# values should be between 0.0 and 1.0
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # set activation threshold for this model

In [ ]:
# instantiate the predictor
# https://detectron2.readthedocs.io/en/latest/modules/engine.html?highlight=defaultpredictor#detectron2.engine.defaults.DefaultPredictor
predictor = DefaultPredictor(cfg)
# get the predictor's outputs
outputs = predictor(im)

In [ ]:

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set activation threshold for this model

# instantiate the predictor
# https://detectron2.readthedocs.io/en/latest/modules/engine.html?highlight=defaultpredictor#detectron2.engine.defaults.DefaultPredictor
predictor = DefaultPredictor(cfg)
# get the predictor's outputs
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

## Inference with Mask RCNN

In [ ]:
# get the YAML file with the model configuration
# https://detectron2.readthedocs.io/en/latest/modules/config.html?highlight=get_cfg#detectron2.config.get_cfg
cfg = get_cfg()

# the set of available config files is on https://github.com/facebookresearch/detectron2/tree/master/configs
# we need to do the calls to both merge_from_file() and cfg.MODEL.WEIGHTS() with the same yaml file specification

cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml'))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml')


# Experiment: change the activation thresholds
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5


# instantiate the predictor
# https://detectron2.readthedocs.io/en/latest/modules/engine.html?highlight=defaultpredictor#detectron2.engine.defaults.DefaultPredictor
predictor = DefaultPredictor(cfg)
predictor

In [ ]:
# get the predictor's outputs
outputs = predictor(im)
outputs

In [ ]:
# check what we can get inference of
# https://detectron2.readthedocs.io/en/latest/modules/structures.html#detectron2.structures.Instances.get_fields
fields = outputs['instances'].get_fields()
fields.keys()

In [ ]:
# We can use detectron's `Visualizer` to draw the predictions on the image.
# https://detectron2.readthedocs.io/en/latest/modules/utils.html?highlight=Visualizer#detectron2.utils.visualizer.Visualizer
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.5)
v

In [ ]:
#https://detectron2.readthedocs.io/en/latest/modules/utils.html?highlight=draw_instance_predictions#detectron2.utils.visualizer.Visualizer.draw_instance_predictions
# Exercise: get rid of the chair
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

## Inference with a keypoint detection model

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config

#experiment with a keypoint detection model
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)

#Q: How do we change the number of detected people?
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6


outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
dir(cfg.MODEL.ROI_HEADS)

## Inference with a panoptic segmentation model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
# We choose a feature pyramid network with 101 layers to perform panoptic segmentation
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# check out stuff_classes and thing_classes
v.metadata

# notice the overlap between stuff and things
print(v.metadata.stuff_classes)

print(v.metadata.thing_classes)

In [ ]:
im_2 = cv2.imread("./input.jpg")
cv2_imshow(im_2)

In [ ]:
panoptic_seg, segments_info = predictor(im_2)["panoptic_seg"]
v = Visualizer(im_2[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
im_3 = cv2.imread("./input2.jpg")
cv2_imshow(im_3)

In [ ]:
panoptic_seg, segments_info = predictor(im_3)["panoptic_seg"]
v = Visualizer(im_3[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])